<a href="https://colab.research.google.com/github/laresamdeola/Machine-Learning/blob/main/Audio_Engineering_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
import os

In [74]:
data_filepaths = os.listdir('/content/sample_data/Audio_Data')

In [82]:
# data_filepaths

In [83]:
data_filepaths_2 = os.getcwd() + '/sample_data/Audio_Data'

In [84]:
data_filepaths_2

'/content/sample_data/Audio_Data'

In [43]:
os.getcwd()

'/content'

In [51]:
# filepaths

In [106]:
# Function to extract features from audio files
def extract_features(filepaths, mfcc=True, chroma=False, mel=False):
  features = []
  for audio_file in filepaths:
      if mfcc:
        data, sample_rate = librosa.load(audio_file, sr=None)
        mfcc_features = librosa.feature.mfcc(y=data, sr=sample_rate)
        mfcc_mean = np.mean(mfcc_features.T, axis=0)
        features.append(mfcc_mean)
      if chroma:
        chroma_features = librosa.feature.chroma_stft(y=audio_file, sr=sample_rate)
        chroma_features = np.mean(chroma_features.T,axis=0)
        features = np.hstack((features, chroma_features))
      if mel:
        mel_features = librosa.feature.melspectrogram(y=audio_file, sr=sample_rate)
        mel_features = np.mean(mel_features.T,axis=0)
        features = np.hstack((features, mel_features))
  return np.array(features)

In [86]:
# extract_features(data_file_paths)

Achieve.MP4
Add.MP4


In [91]:
to_ignore = ['Region.MP4', 'Fight.m4a', 'Fear.MP4', 'Hurry.m4a', 'Ration.MP4', 'Decide.m4a', 'Bill.m4a', 'Race.MP4', 'Ready.MP4']

In [107]:
# Function to prepare dataset
def prepare_dataset(filepaths):
    features, labels = [], []
    for file_path in filepaths:
        # Extract features
        if file_path in to_ignore:
          continue
        extracted_features = extract_features([file_path])
        features.append(extracted_features)
        # Assign label based on filename (assuming filename contains label)
        if 'm4a' in file_path:
          label = 'Noise'
        else:
          label = 'Truth'
        # label = file_path.split('/')[-1].split('-')[0]
        labels.append(label)
    return np.array(features), np.array(labels)

In [108]:
data_file_paths = ['Achieve.MP4', 'Add.MP4']

all_file_paths = [os.getcwd() + '/sample_data/Audio_Data/' + i for i in data_filepaths]

In [109]:
X, y = prepare_dataset(all_file_paths)

<ipython-input-106-1a2f0ff95a04>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-106-1a2f0ff95a04>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-106-1a2f0ff95a04>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/loca

In [110]:
X.ndim

3

In [111]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
X_train.ndim

3

In [113]:
X_train_flat = np.reshape(X_train, (X_train.shape[0], -1))
X_test_flat = np.reshape(X_test, (X_test.shape[0], -1))

In [114]:
X_train_flat.ndim

2

In [125]:
# Convert string labels to binary labels
y_train_binary = np.where(y_train == "truth", 1, 0)
y_test_binary = np.where(y_test == "truth", 1, 0)

In [126]:
# Define and train the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_flat.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [127]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1344      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [130]:
model.fit(X_train_flat, y_train_binary, epochs=10, batch_size=32, validation_data=(X_test_flat, y_test_binary))

Epoch 1/10
5/5 [==============================] - 0s 56ms/step - loss: 1.8004e-13 - accuracy: 1.0000 - val_loss: 3.8293e-17 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 0s 26ms/step - loss: 1.6808e-13 - accuracy: 1.0000 - val_loss: 3.6601e-17 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 0s 25ms/step - loss: 1.6177e-13 - accuracy: 1.0000 - val_loss: 3.5603e-17 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 21ms/step - loss: 1.5982e-13 - accuracy: 1.0000 - val_loss: 3.5006e-17 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 28ms/step - loss: 1.5703e-13 - accuracy: 1.0000 - val_loss: 3.4647e-17 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 0s 26ms/step - loss: 1.5565e-13 - accuracy: 1.0000 - val_loss: 3.4430e-17 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 0s 26ms/step - loss: 1.5520e-13 - accuracy: 1.0000 - val_loss: 3.429

In [115]:
# Define and train the Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train_flat, y_train)
y_pred = clf.predict(X_test_flat)

# Calculate accuracy for Decision Tree classifier
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy:", accuracy)

Decision Tree Accuracy: 1.0


In [116]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_flat, y_train)
rf_predictions = rf_classifier.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.975


In [131]:
from sklearn.naive_bayes import GaussianNB

# Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_flat, y_train)
nb_predictions = nb_classifier.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

Naive Bayes Accuracy: 0.975


In [132]:
from sklearn.svm import SVC

# Support Vector Machine (SVM) Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_flat, y_train)
svm_predictions = svm_classifier.predict(X_test_flat)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 1.0


3

1